# Test a Stacked Ensemble

1. LGBM Classifier (5 Outputs) [Do HPT]
2. LGBM Regressor (1 Output) [Do HPT]
3. LGBM Regressor with encodings (1 Output) [Do HPT]
4. LGBM Regressor with reduced features (1 Output) [Do HPT]
5. XGB Classifier (5 Outputs) [Do HPT]
6. XGB Regressor (1 Ouput) [Do HPT]
7. XGB Regressor with encodings (1 Output) [Do HPT]
8. XGB Regressor with reduced features (1 Output) [Do HPT]
9. Logistic Regression (5 Outputs)
10. Linear Regression (1 Output)

Finally, use a neural net with pearson scoring OR Linear regression (with or without original dataset)

# Set up

In [2]:
!git clone --recursive https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 19012 (delta 10), reused 25 (delta 3), pack-reused 18962
Receiving objects: 100% (19012/19012), 14.90 MiB | 22.70 MiB/s, done.
Resolving deltas: 100% (13864/13864), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'


In [3]:
%cd /content/LightGBM

/content/LightGBM


In [4]:
!mkdir build

In [5]:
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--

In [6]:
%cd /content/LightGBM/python-package

/content/LightGBM/python-package


In [7]:
!sudo python setup.py install --precompile

running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
reading man

# Download Dataset

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/Colab Notebooks/Numerai

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/Numerai'
/content/LightGBM/python-package


In [9]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import cross_val_predict
from IPython.display import clear_output

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [10]:
train_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Numerai/Data/final_training_data.csv')
val_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Numerai/Data/final_validation_data.csv')

TARGET_NAME = f"target_kazutsugi"
PREDICTION_NAME = f"prediction"
feature_names = [f for f in train_df.columns if f.startswith("feature")]
reduced_feature_names = [f for f in train_df.columns if f.startswith("reduced")]
xgb_feature_names = [f for f in train_df.columns if (f.startswith("feature") or f.startswith("xgb"))]
lgbm_feature_names = [f for f in train_df.columns if (f.startswith("feature") or f.startswith("lgbm"))]

train_df[feature_names] = train_df[feature_names].astype(np.float16)
train_df[TARGET_NAME] = train_df[TARGET_NAME].astype(np.float16)
val_df[feature_names] = val_df[feature_names].astype(np.float16)
val_df[TARGET_NAME] = val_df[TARGET_NAME].astype(np.float16)

In [11]:
def score(df, PREDICTION_NAME='prediction'):
    pct_ranks = df[PREDICTION_NAME].rank(pct=True, method="first")
    targets = df[TARGET_NAME]
    return np.corrcoef(targets, pct_ranks)[0, 1]

In [12]:
print(train_df.shape)
train_df.head()

(501808, 542)


,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,reduced_feature_188,reduced_feature_189,reduced_feature_190,reduced_feature_191,reduced_feature_192,reduced_feature_193,reduced_feature_194,reduced_feature_195,reduced_feature_196,reduced_feature_197,reduced_feature_198,reduced_feature_199,xgb_feature_0,xgb_feature_1,xgb_feature_2,xgb_feature_3,xgb_feature_4,xgb_feature_5,xgb_feature_6,xgb_feature_7,xgb_feature_8,xgb_feature_9,xgb_feature_10,xgb_feature_11,xgb_feature_12,xgb_feature_13,xgb_feature_14,xgb_feature_15,xgb_feature_16,xgb_feature_17,xgb_feature_18,lgbm_feature_0,lgbm_feature_1,lgbm_feature_2,lgbm_feature_3,lgbm_feature_4,lgbm_feature_5,lgbm_feature_6,lgbm_feature_7,lgbm_feature_8
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.5,1.0,0.5,0.00,0.50,0.50,0.00,0.0,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.5,0.50,0.75,0.5,...,0.3750,1.000,0.25,0.625,0.500,0.25,0.125,0.25,0.875,0.00,0.500,0.250,-0.349383,-1.441544,-1.784221,-0.069848,1.732101,-0.401702,-0.811690,0.619830,0.525866,0.544893,0.518023,0.364337,0.273330,0.612157,0.584791,0.704958,0.429479,0.397570,0.537267,-0.349382,-1.441543,-1.784219,0.303880,0.216762,0.382686,0.445277,0.478433,0.246845
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.5,0.5,0.5,0.50,0.25,0.25,0.50,0.0,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.5,0.25,0.75,0.5,...,0.5000,0.250,1.00,0.875,0.375,0.75,0.750,0.75,0.125,0.75,0.750,1.000,-0.695059,-2.199356,0.393313,-0.565352,-0.887640,1.098592,-0.504342,-1.579269,0.430178,0.328498,0.433338,0.337190,0.297101,0.844717,0.571642,0.449587,0.421337,0.701694,0.667045,-0.695060,-2.199350,0.393300,0.553677,0.248747,0.650030,0.600467,0.637539,0.315280
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.5,0.5,1.0,0.50,0.50,0.50,0.25,0.0,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.5,0.50,0.75,0.5,...,0.5625,1.000,0.00,0.250,0.625,0.25,0.375,0.25,0.000,0.00,0.000,0.250,0.907658,-0.358365,-0.935118,0.446002,-1.227900,-1.925119,-0.954609,-0.624884,0.679789,0.659405,0.371719,0.316489,0.563163,0.581140,0.512820,0.625947,0.320122,0.685917,0.678815,0.907658,-0.358362,-0.935140,0.392139,0.438776,0.405294,0.550879,0.508910,0.280421
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.0,0.5,0.5,0.00,1.00,0.00,0.75,0.0,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,0.9375,0.500,0.25,0.375,1.000,0.25,0.125,0.75,0.625,0.25,0.375,0.250,1.598843,-1.013379,-1.560695,-2.089551,-0.163186,-0.796688,-0.747240,-0.019883,0.465539,0.477262,0.366082,0.443915,0.343933,0.658179,0.240033,0.704373,0.337026,0.553835,0.476238,1.598843,-1.013379,-1.560685,0.377822,0.453177,0.639378,0.510745,0.529151,0.255674
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.0,0.5,0.5,0.25,0.00,0.50,0.00,0.5,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,0.8750,0.125,1.00,1.000,0.000,0.75,0.250,1.00,0.000,0.00,1.000,0.125,-0.088821,-1.741690,-0.763098,-0.403356,-0.009025,1.188055,-0.386061,0.864460,0.443174,0.356642,0.531192,0.343516,0.353477,0.676516,0.428447,0.58324

In [13]:
print(val_df.shape)
val_df.head()

(106895, 542)


,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,reduced_feature_188,reduced_feature_189,reduced_feature_190,reduced_feature_191,reduced_feature_192,reduced_feature_193,reduced_feature_194,reduced_feature_195,reduced_feature_196,reduced_feature_197,reduced_feature_198,reduced_feature_199,xgb_feature_0,xgb_feature_1,xgb_feature_2,xgb_feature_3,xgb_feature_4,xgb_feature_5,xgb_feature_6,xgb_feature_7,xgb_feature_8,xgb_feature_9,xgb_feature_10,xgb_feature_11,xgb_feature_12,xgb_feature_13,xgb_feature_14,xgb_feature_15,xgb_feature_16,xgb_feature_17,xgb_feature_18,lgbm_feature_0,lgbm_feature_1,lgbm_feature_2,lgbm_feature_3,lgbm_feature_4,lgbm_feature_5,lgbm_feature_6,lgbm_feature_7,lgbm_feature_8
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,0.25,0.50,0.5,0.25,0.0,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.0,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,...,0.2500,0.375,0.25,0.25,1.00,1.000,1.000,1.000,0.125,0.625,0.250,0.75,0.120206,1.551120,1.712235,-0.669770,-2.248873,1.187683,0.441092,-0.430071,0.289276,0.384547,0.382106,0.385016,0.539111,0.654335,0.248370,0.184573,0.533881,0.622801,0.627099,0.120206,1.551119,1.712246,0.664974,0.503687,0.544194,0.665161,0.432860,0.257939
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,0.75,0.25,0.0,0.75,0.5,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.0,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,...,0.0000,0.000,1.00,0.25,1.00,0.250,0.000,0.875,0.000,0.000,0.250,0.00,2.527598,0.106757,-1.088938,-0.823978,1.176287,2.246339,1.333658,0.551338,0.331852,0.401299,0.739960,0.392725,0.246025,0.474380,0.344525,0.596060,0.515408,0.624127,0.694898,2.527598,0.106757,-1.088935,0.504223,0.414093,0.620324,0.381888,0.478437,0.410321
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,1.00,0.50,0.5,1.00,1.0,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.5,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,...,0.0000,0.250,0.75,1.00,0.50,0.500,1.000,0.000,0.500,1.000,1.000,1.00,-1.336355,3.505574,1.418149,0.621056,0.693367,-0.194254,1.031963,-2.353089,0.201501,0.313064,0.394394,0.268587,0.478782,0.345278,0.526970,0.456296,0.790289,0.574499,0.667246,-1.336355,3.505574,1.418144,0.471273,0.398253,0.035018,0.535121,0.219912,0.180064
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,0.50,0.75,1.0,0.75,0.0,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.0,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,...,0.3125,0.875,0.75,0.00,0.50,0.875,0.875,0.875,0.750,0.250,0.000,1.00,-1.564395,3.155077,0.842485,0.456792,-1.635734,1.184766,0.369005,1.118786,0.223846,0.455838,0.455837,0.437610,0.549473,0.693071,0.369459,0.320194,0.513130,0.401977,0.691888,-1.564395,3.155080,0.842473,0.671360,0.414074,0.448732,0.564994,0.389353,0.192545
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,0.25,0.25,0.5,0.50,1.0,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.0,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,...,1.0000,0.000,0.00,0.75,0.75,0.375,0.250,0.625,0.750,0.375,0.625,0.00,0.931153,0.517618,-0.132495,-0.730126,0.158312,-0.378191,-2.346678,-0.189877,0.540932,0.586559,0.465905,0.443415,0.516665,0.

# Training Functions

In [14]:
def train_model_with_cv(model, X_train, y_train, pred_name, folds=10, kind='reg'):
  if kind == 'reg':

    train_df[pred_name] = cross_val_predict(model, X_train, y_train, cv=folds)
    clear_output(wait=True)

    train_correlations = train_df.groupby("era").apply(lambda x: score(x, pred_name))
    string = f"Model: {pred_name}, Train correlations: mean {train_correlations.mean()} and std {train_correlations.std()}"
  
  else:

    y_proba = cross_val_predict(model, X_train, y_train, cv=folds, method='predict_proba')
    train_df[f'{pred_name}_1'] = y_proba.argmax(axis=1)/4
    clear_output(wait=True)

    train_correlations = train_df.groupby("era").apply(lambda x: score(x, f'{pred_name}_1'))
    string = f"Model: {pred_name}, Train correlations: mean {train_correlations.mean()} and std {train_correlations.std()}"

    train_df[f'{pred_name}_1'], train_df[f'{pred_name}_2'], train_df[f'{pred_name}_3'], train_df[f'{pred_name}_4'], train_df[f'{pred_name}_5'] = y_proba[:,0], y_proba[:,1], y_proba[:,2], y_proba[:,3], y_proba[:,4]

  return string

In [15]:
# model_list in the form [{'model': model, 'pred_name': name, 'kind': 'reg'/'class'}, {'model': model, 'pred_name': name, 'kind': 'reg'/'class'}]

def train_all_models(model_list, X_train, y_train, folds):
  string_ls = []
  for model_dict in model_list:
    string = train_model_with_cv(model_dict['model'], X_train, y_train, model_dict['pred_name'], folds=10, kind=model_dict['kind'])
    string_ls.append(string)

    for string in string_ls:
      print(string)

# Train Models

In [20]:
from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [17]:
params = {}
params['lgbreg'] = {'boosting_type': 'gbdt', 'device_type': 'gpu', 'metric': 'l2', 'silent': True, 'n_estimators': 410, 'learning_rate': 0.0185, 'num_leaves': 164, 'max_depth': 10, 'min_child_samples': 4000, 'colsample_bytree': 0.48, 'max_bin': 280}
params['lgbclass'] = {'boosting_type': 'gbdt', 'device_type': 'gpu', 'silent': True, 'n_estimators': 175, 'learning_rate': 0.046, 'num_leaves': 128, 'max_depth': 9, 'min_child_samples': 2750, 'colsample_bytree': 0.31, 'max_bin': 650}
params['xgbreg'] = {'booster': 'gbtree', 'tree_method': 'gpu_hist', 'verbosity': 0, 'n_estimators': 2540, 'learning_rate': 0.0037, 'gamma': 0.0193, 'max_depth': 7, 'min_child_weight': 2.89, 'subsample': 0.71, 'colsample_bytree': 0.28, 'lambda': 0.0367, 'max_bin': 970}
params['xgbclass'] = {'booster': 'gbtree', 'tree_method': 'gpu_hist', 'verbosity': 0, 'n_estimators': 900, 'learning_rate': 0.0276, 'gamma': 0.23, 'max_depth': 5, 'min_child_weight': 2.2, 'subsample': 0.44, 'colsample_bytree': 0.38, 'lambda': 0.0123, 'max_bin': 720}

In [18]:
model_list = []
model_list.append({'model': LGBMRegressor(**params['lgbreg']), 'pred_name': 'cv_lgbreg1_out', 'kind': 'reg'})
model_list.append({'model': LGBMClassifier(**params['lgbclass']), 'pred_name': 'cv_lgbclass1_out', 'kind': 'class'})
model_list.append({'model': XGBRegressor(**params['xgbreg']), 'pred_name': 'cv_xgbreg1_out', 'kind': 'reg'})
model_list.append({'model': XGBClassifier(**params['xgbclass']), 'pred_name': 'cv_xgbclass1_out', 'kind': 'class'})
model_list.append({'model': LinearRegression(), 'pred_name': 'cv_linreg1_out', 'kind': 'reg'})
model_list.append({'model': LogisticRegression(), 'pred_name': 'cv_logreg1_out', 'kind': 'class'})

In [19]:
train_all_models(model_list, train_df[feature_names], train_df[TARGET_NAME], 10)

Model: cv_lgbreg1_out, Train correlations: mean 0.05382182900424805 and std 0.03645937457318164
Model: cv_lgbclass1_out, Train correlations: mean 0.033340972945850054 and std 0.02716102481582032
Model: cv_xgbreg1_out, Train correlations: mean 0.05560443610876123 and std 0.03740884435946288
Model: cv_xgbclass1_out, Train correlations: mean 0.03454445107862647 and std 0.02628143000896918
Model: cv_linreg1_out, Train correlations: mean 0.03915657263523684 and std 0.03664291262823588
Model: cv_logreg1_out, Train correlations: mean 0.025547635588300067 and std 0.028270720478147014


In [20]:
model_list_red = []
model_list_red.append({'model': LGBMRegressor(**params['lgbreg']), 'pred_name': 'cv_lgbreg2_out', 'kind': 'reg'})
model_list_red.append({'model': XGBRegressor(**params['xgbreg']), 'pred_name': 'cv_xgbreg2_out', 'kind': 'reg'})

In [21]:
train_all_models(model_list_red, train_df[reduced_feature_names], train_df[TARGET_NAME], 10)

Model: cv_lgbreg2_out, Train correlations: mean 0.05376659757278124 and std 0.03671614010102047
Model: cv_xgbreg2_out, Train correlations: mean 0.0549691360053004 and std 0.03725735088269433


In [22]:
model_list_lgbm = []
new_param = {'boosting_type': 'gbdt', 'device_type': 'gpu', 'metric': 'l2', 'silent': True, 'n_estimators': 410, 'learning_rate': 0.0185, 'num_leaves': 164, 'max_depth': 10, 'min_child_samples': 4000, 'colsample_bytree': 0.48, 'max_bin': 100}
model_list_lgbm.append({'model': LGBMRegressor(**new_param), 'pred_name': 'cv_lgbreg3_out', 'kind': 'reg'})

In [23]:
train_all_models(model_list_lgbm, train_df[lgbm_feature_names], train_df[TARGET_NAME], 10)

Model: cv_lgbreg3_out, Train correlations: mean 0.05430045448791592 and std 0.03671868227579534


In [24]:
model_list_xgb = []
model_list_xgb.append({'model': XGBRegressor(**params['xgbreg']), 'pred_name': 'cv_xgbreg3_out', 'kind': 'reg'})

In [25]:
train_all_models(model_list_xgb, train_df[xgb_feature_names], train_df[TARGET_NAME], 10)

Model: cv_xgbreg3_out, Train correlations: mean 0.05352164123463375 and std 0.03714723866658102


In [26]:
train_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Numerai/Data/train_df_with_predictions.csv', index=False)

# Meta Estimator

In [14]:
train_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Numerai/Data/train_df_with_predictions.csv')
train_df.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,xgb_feature_10,xgb_feature_11,xgb_feature_12,xgb_feature_13,xgb_feature_14,xgb_feature_15,xgb_feature_16,xgb_feature_17,xgb_feature_18,lgbm_feature_0,lgbm_feature_1,lgbm_feature_2,lgbm_feature_3,lgbm_feature_4,lgbm_feature_5,lgbm_feature_6,lgbm_feature_7,lgbm_feature_8,cv_lgbreg1_out,cv_lgbclass1_out_1,cv_lgbclass1_out_2,cv_lgbclass1_out_3,cv_lgbclass1_out_4,cv_lgbclass1_out_5,cv_xgbreg1_out,cv_xgbclass1_out_1,cv_xgbclass1_out_2,cv_xgbclass1_out_3,cv_xgbclass1_out_4,cv_xgbclass1_out_5,cv_linreg1_out,cv_logreg1_out_1,cv_logreg1_out_2,cv_logreg1_out_3,cv_logreg1_out_4,cv_logreg1_out_5,cv_lgbreg2_out,cv_xgbreg2_out,cv_lgbreg3_out,cv_xgbreg3_out
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.5,1.0,0.5,0.00,0.50,0.50,0.00,0.0,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.5,0.50,0.75,0.5,...,0.518023,0.364337,0.273330,0.612157,0.584791,0.704958,0.429479,0.397570,0.537267,-0.349382,-1.441543,-1.784219,0.303880,0.216762,0.382686,0.445277,0.478433,0.246845,0.500072,0.133846,0.236144,0.264938,0.241343,0.123730,0.485541,0.130857,0.249920,0.261901,0.228610,0.128711,0.499150,0.096837,0.245549,0.315365,0.247629,0.094620,0.504378,0.483165,0.496039,0.487033
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.5,0.5,0.5,0.50,0.25,0.25,0.50,0.0,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.5,0.25,0.75,0.5,...,0.433338,0.337190,0.297101,0.844717,0.571642,0.449587,0.421337,0.701694,0.667045,-0.695060,-2.199350,0.393300,0.553677,0.248747,0.650030,0.600467,0.637539,0.315280,0.500678,0.151117,0.236570,0.263261,0.205274,0.143778,0.498782,0.163131,0.227736,0.231030,0.205840,0.172264,0.523520,0.142478,0.221591,0.239235,0.223280,0.173417,0.497172,0.490947,0.497153,0.506480
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.5,0.5,1.0,0.50,0.50,0.50,0.25,0.0,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.5,0.50,0.75,0.5,...,0.371719,0.316489,0.563163,0.581140,0.512820,0.625947,0.320122,0.685917,0.678815,0.907658,-0.358362,-0.935140,0.392139,0.438776,0.405294,0.550879,0.508910,0.280421,0.465989,0.171429,0.204801,0.285159,0.215755,0.122856,0.480088,0.163716,0.224433,0.266364,0.215344,0.130143,0.482267,0.141739,0.249415,0.270759,0.205451,0.132635,0.470566,0.475935,0.482908,0.482425
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.0,0.5,0.5,0.00,1.00,0.00,0.75,0.0,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,0.366082,0.443915,0.343933,0.658179,0.240033,0.704373,0.337026,0.553835,0.476238,1.598843,-1.013379,-1.560685,0.377822,0.453177,0.639378,0.510745,0.529151,0.255674,0.529036,0.208186,0.177870,0.153156,0.183946,0.276842,0.514985,0.216631,0.179622,0.151096,0.183817,0.268834,0.510642,0.213763,0.183717,0.184239,0.181499,0.236782,0.515534,0.517889,0.514829,0.504893
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.0,0.5,0.5,0.25,0.00,0.50,0.00,0.5,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,0.531192,0.343516,0.353477,0.676516,0.428447,0.583248,0.481688,0.516346,0.502558,-0.08

In [15]:
layer_0_names = [f for f in train_df.columns if f.startswith("cv")]
layer_0_names

['cv_lgbreg1_out',
 'cv_lgbclass1_out_1',
 'cv_lgbclass1_out_2',
 'cv_lgbclass1_out_3',
 'cv_lgbclass1_out_4',
 'cv_lgbclass1_out_5',
 'cv_xgbreg1_out',
 'cv_xgbclass1_out_1',
 'cv_xgbclass1_out_2',
 'cv_xgbclass1_out_3',
 'cv_xgbclass1_out_4',
 'cv_xgbclass1_out_5',
 'cv_linreg1_out',
 'cv_logreg1_out_1',
 'cv_logreg1_out_2',
 'cv_logreg1_out_3',
 'cv_logreg1_out_4',
 'cv_logreg1_out_5',
 'cv_lgbreg2_out',
 'cv_xgbreg2_out',
 'cv_lgbreg3_out',
 'cv_xgbreg3_out']

In [21]:
meta_model_lin = LinearRegression()
meta_model_lin.fit(train_df[layer_0_names], train_df[TARGET_NAME])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
train_df[PREDICTION_NAME] = meta_model_lin.predict(train_df[layer_0_names])

train_correlations = train_df.groupby("era").apply(score)
print(f"On training the correlation has mean {train_correlations.mean()} and std {train_correlations.std()}")

On training the correlation has mean 0.056477450590007014 and std 0.03796051164467176


In [23]:
meta_model_xgb = XGBRegressor()
meta_model_xgb.fit(train_df[layer_0_names], train_df[TARGET_NAME])

[00:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [24]:
train_df[PREDICTION_NAME] = meta_model_xgb.predict(train_df[layer_0_names])

train_correlations = train_df.groupby("era").apply(score)
print(f"On training the correlation has mean {train_correlations.mean()} and std {train_correlations.std()}")

On training the correlation has mean 0.06724744201013608 and std 0.03749032362852529


# Prediction Functions

In [33]:
def train_model_for_prediction(model, X_train, y_train, X_val, pred_name, kind='reg'):

  if kind == 'reg':

    model.fit(X_train, y_train)
    val_df[pred_name] = model.predict(X_val)
    clear_output(wait=True)

    validation_correlations = val_df.groupby("era").apply(lambda x: score(x, pred_name))
    string = f"Model: {pred_name}, Val correlations: mean {validation_correlations.mean()} and std {validation_correlations.std()}"
  
  else:

    model.fit(X_train, (y_train*4).astype(int))
    val_df[f'{pred_name}_1'] = model.predict(X_val)/4
    clear_output(wait=True)

    validation_correlations = val_df.groupby("era").apply(lambda x: score(x, f'{pred_name}_1'))
    string = f"Model: {pred_name}, Val correlations: mean {validation_correlations.mean()} and std {validation_correlations.std()}"

    y_proba = model.predict_proba(X_val)
    val_df[f'{pred_name}_1'], val_df[f'{pred_name}_2'], val_df[f'{pred_name}_3'], val_df[f'{pred_name}_4'], val_df[f'{pred_name}_5'] = y_proba[:,0], y_proba[:,1], y_proba[:,2], y_proba[:,3], y_proba[:,4]

  return string

In [34]:
# model_list in the form [{'model': model, 'pred_name': name, 'kind': 'reg'/'class'}, {'model': model, 'pred_name': name, 'kind': 'reg'/'class'}]

def train_all_models_prediction(model_list, X_train, y_train, X_val):
  string_ls = []
  for model_dict in model_list:
    string = train_model_for_prediction(model_dict['model'], X_train, y_train, X_val, model_dict['pred_name'], kind=model_dict['kind'])
    string_ls.append(string)

    for string in string_ls:
      print(string)

# Prediction Models

In [35]:
model_list = []
model_list.append({'model': LGBMRegressor(**params['lgbreg']), 'pred_name': 'pred_lgbreg1_out', 'kind': 'reg'})
model_list.append({'model': LGBMClassifier(**params['lgbclass']), 'pred_name': 'pred_lgbclass1_out', 'kind': 'class'})
model_list.append({'model': XGBRegressor(**params['xgbreg']), 'pred_name': 'pred_xgbreg1_out', 'kind': 'reg'})
model_list.append({'model': XGBClassifier(**params['xgbclass']), 'pred_name': 'pred_xgbclass1_out', 'kind': 'class'})
model_list.append({'model': LinearRegression(), 'pred_name': 'pred_linreg1_out', 'kind': 'reg'})
model_list.append({'model': LogisticRegression(), 'pred_name': 'pred_logreg1_out', 'kind': 'class'})

In [36]:
train_all_models_prediction(model_list, train_df[feature_names], train_df[TARGET_NAME], val_df[feature_names])

Model: pred_lgbreg1_out, Val correlations: mean 0.029827307607557287 and std 0.02960916332395143
Model: pred_lgbclass1_out, Val correlations: mean 0.015310941805131102 and std 0.02215503944126782
Model: pred_xgbreg1_out, Val correlations: mean 0.03059556834150733 and std 0.03074756446806095
Model: pred_xgbclass1_out, Val correlations: mean 0.018770583888729774 and std 0.02286552533960343
Model: pred_linreg1_out, Val correlations: mean 0.01673990739301305 and std 0.035834689364694854
Model: pred_logreg1_out, Val correlations: mean 0.0130701841545852 and std 0.02511224469834383


In [37]:
model_list_red = []
model_list_red.append({'model': LGBMRegressor(**params['lgbreg']), 'pred_name': 'pred_lgbreg2_out', 'kind': 'reg'})
model_list_red.append({'model': XGBRegressor(**params['xgbreg']), 'pred_name': 'pred_xgbreg2_out', 'kind': 'reg'})

In [38]:
train_all_models_prediction(model_list_red, train_df[reduced_feature_names], train_df[TARGET_NAME], val_df[reduced_feature_names])

Model: pred_lgbreg2_out, Val correlations: mean 0.028683314332101053 and std 0.030976404217967695
Model: pred_xgbreg2_out, Val correlations: mean 0.028715759240777718 and std 0.03094873947834988


In [67]:
model_list_lgbm = []
new_param = {'boosting_type': 'gbdt', 'device_type': 'gpu', 'metric': 'l2', 'silent': True, 'n_estimators': 410, 'learning_rate': 0.0185, 'num_leaves': 164, 'max_depth': 10, 'min_child_samples': 4000, 'colsample_bytree': 0.48, 'max_bin': 100}
model_list_lgbm.append({'model': LGBMRegressor(**new_param), 'pred_name': 'pred_lgbreg3_out', 'kind': 'reg'})

In [ ]:
train_all_models_prediction(model_list_lgbm, train_df[lgbm_feature_names], train_df[TARGET_NAME], val_df[lgbm_feature_names])

In [ ]:
model_list_xgb = []
model_list_xgb.append({'model': XGBRegressor(**params['xgbreg']), 'pred_name': 'pred_xgbreg3_out', 'kind': 'reg'})

In [ ]:
train_all_models_prediction(model_list_xgb, train_df[xgb_feature_names], train_df[TARGET_NAME], val_df[xgb_feature_names])

# Meta-model Prediction

In [ ]:
val_layer_0_names = [f for f in val_df.columns if f.startswith("pred")]
val_layer_0_names

['pred_lgbreg1_out',
 'pred_lgbclass1_out_1',
 'pred_lgbclass1_out_2',
 'pred_lgbclass1_out_3',
 'pred_lgbclass1_out_4',
 'pred_lgbclass1_out_5',
 'pred_xgbreg1_out',
 'pred_xgbclass1_out_1',
 'pred_xgbclass1_out_2',
 'pred_xgbclass1_out_3',
 'pred_xgbclass1_out_4',
 'pred_xgbclass1_out_5',
 'pred_linreg1_out',
 'pred_logreg1_out_1',
 'pred_logreg1_out_2',
 'pred_logreg1_out_3',
 'pred_logreg1_out_4',
 'pred_logreg1_out_5',
 'pred_lgbreg2_out',
 'pred_xgbreg2_out',
 'pred_lgbreg3_out',
 'pred_xgbreg3_out']

In [ ]:
val_df[PREDICTION_NAME] = meta_model_lin.predict(val_df[val_layer_0_names])

validation_correlations = val_df.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and std {validation_correlations.std()}")

On validation the correlation has mean 0.026998836633248126 and std 0.03248055826952012


In [ ]:
val_df[PREDICTION_NAME] = meta_model_xgb.predict(val_df[val_layer_0_names])

validation_correlations = val_df.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and std {validation_correlations.std()}")